In [ ]:
import pandas as pd
from typing import Literal
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import matplotlib.pyplot as plt
    
from utils.loader import MovieLensDataset
from arquitecture.Recommender import Recommender_2
import torch.optim as optim

SEED = 55
BATCH = 300
NUN_THREADS = 6
DEVICE = "cuda"
NUM_EPOCH = 100
LEARNING_RATE = 0.0005


/home/lingfeng/Desktop/venvs/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = MovieLensDataset(ml_path="ml-100k", split="train", transpose_ratings=True, seed=SEED)
test_dataset = MovieLensDataset(ml_path="ml-100k", split="test", transpose_ratings=True, seed=SEED)
val_dataset = MovieLensDataset(ml_path="ml-100k", split="val", transpose_ratings=True, seed=SEED)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)

for user_data_tensor, rating_train_tensor, rating_test_tensor in train_dataloader:
    print("Rating Tensor Shape:", rating_train_tensor.size())
    print("User Data Tensor Shape:", user_data_tensor.size())
    print(f"Max index in rating tensor: {rating_train_tensor.max().item()}")
    print(f"Min index in rating tensor: {rating_train_tensor.min().item()}")
    
    # Get true num_ratings from data
    batch_size, num_ratings_actual, rating_size = rating_train_tensor.size()
    user_batch_size, user_data_dim = user_data_tensor.size()
    
    print(f"Actual num_ratings from tensor: {num_ratings_actual}")
    print(f"Actual user_data_input_dim: {user_data_dim}")
    break

Rating Tensor Shape: torch.Size([300, 22, 19])
User Data Tensor Shape: torch.Size([300, 23])
Max index in rating tensor: 100
Min index in rating tensor: 0
Actual num_ratings from tensor: 22
Actual user_data_input_dim: 23


In [ ]:
def create_model(trial):
    # Hyperparameters to optimize
    ratings_embedding_dim = trial.suggest_int('ratings_embedding_dim', 4, 32)
    ratings_lstm_hidden_size = trial.suggest_int('ratings_lstm_hidden_size', 8, 64)
    ratings_lstm_num_layers = trial.suggest_int('ratings_lstm_num_layers', 1, 32)
    ratings_word_size = trial.suggest_int('ratings_word_size', 8, 32)
    ratings_final_mlp_factor = trial.suggest_int('ratings_final_mlp_factor', 2, 16)
    ratings_embedding_output = trial.suggest_int('ratings_embedding_output', 16, 64)
    
    user_embedding_dim = trial.suggest_int('user_embedding_dim', 8, 32)
    user_embedding_output = trial.suggest_int('user_embedding_output', 8, 32)
    user_factor = trial.suggest_int('user_factor', 2, 16)
    
    final_output_size = 19  # Fixed, since this depends on your target size
    expert_factor = trial.suggest_int('expert_factor', 2, 16)
    
    # Create model with suggested hyperparameters
    max_index_in_data = rating_train_tensor.max().item()
    ratings_num_embeddings = max_index_in_data + 1  # Add 1 since indices are 0-based
    
    model = Recommender_2(
        # For ratings embedder
        ratings_num_embeddings=ratings_num_embeddings,
        ratings_embedding_dim=ratings_embedding_dim,
        ratings_num_ratings=num_ratings_actual,
        ratings_lstm_hidden_size=ratings_lstm_hidden_size,
        ratings_lstm_num_layers=ratings_lstm_num_layers,
        ratings_word_size=ratings_word_size,
        ratings_final_mlp_factor=ratings_final_mlp_factor,
        ratings_embedding_output=ratings_embedding_output,
        # For user embedder
        user_num_embeddings=100,  # Could be optimized too if variable
        user_embedding_dim=user_embedding_dim,
        user_embedding_output=user_embedding_output,
        user_data_input_dim=user_data_dim,
        user_factor=user_factor,
        # Output layer
        final_output_size=final_output_size,
        expert_factor=expert_factor
    ).to(DEVICE)
    
    return model

def objective(trial):
    # Create model with hyperparameters suggested by Optuna
    model = create_model(trial)
    
    # Print model parameters for reference
    num_params = sum(p.numel() for p in model.parameters())
    print(f"Trial {trial.number} - Model parameters: {num_params}")
    
    # Train and evaluate
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.MSELoss(reduction="sum")
    
    # Track best validation loss
    best_val_loss = float('inf')
    
    for epoch in range(NUM_EPOCH):
        # --- Training Phase ---
        model.train()
        running_train_loss = 0.0
        for user_data_tensor, rating_train_tensor, rating_test_tensor in train_dataloader:
            optimizer.zero_grad()
            outputs = model(rating_train_tensor.to(DEVICE), user_data_tensor.to(DEVICE)).to(DEVICE)
            loss = criterion(outputs.to(DEVICE), rating_test_tensor.to(DEVICE))
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * rating_test_tensor.size()[1]
        
        epoch_train_loss = running_train_loss / len(train_dataloader.dataset)
        
        # --- Validation Phase ---
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for user_data_tensor, rating_train_tensor, rating_test_tensor in val_dataloader:
                outputs = model(rating_train_tensor.to(DEVICE), user_data_tensor.to(DEVICE))
                loss = criterion(outputs, rating_test_tensor.to(DEVICE))
                running_val_loss += loss.item() * rating_test_tensor.size()[1]
        
        epoch_val_loss = running_val_loss / len(val_dataloader.dataset)
        
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
        
        print(f"Trial {trial.number}, Epoch [{epoch+1}/{NUM_EPOCH}] - "
              f"Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")
        
        # Report intermediate values to Optuna
        trial.report(epoch_val_loss, epoch)
        
        # Handle pruning (early stopping if this trial isn't promising)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return best_val_loss

def evaluate(model: Recommender_2):
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.MSELoss(reduction="sum")
    
    for epoch in range(NUM_EPOCH):
        # --- Training Phase ---
        model.train()  # Set model to training mode
        running_train_loss = 0.0
        
        for user_data_tensor, rating_train_tensor, rating_test_tensor in train_dataloader:
            optimizer.zero_grad()
            
            outputs = model(rating_train_tensor.to(DEVICE), user_data_tensor.to(DEVICE)).to(DEVICE)        
            loss = criterion(outputs.to(DEVICE), rating_test_tensor.to(DEVICE)) 
            
            loss.backward()                 # Backpropagation
            optimizer.step()                # Update model parameters
            running_train_loss += loss.item() * rating_test_tensor.size()[1]
        
        epoch_train_loss = running_train_loss / len(train_dataloader.dataset)

        # --- Validation Phase ---32
        model.eval()  # Set model to evaluation mode
        running_val_loss = 0.0
        with torch.no_grad():
            for user_data_tensor, rating_train_tensor, rating_test_tensor in val_dataloader:
                
                outputs = model(rating_train_tensor.to(DEVICE), user_data_tensor.to(DEVICE))        
                loss = criterion(outputs, rating_test_tensor.to(DEVICE)) 
                
                running_val_loss += loss.item() * rating_test_tensor.size()[1]
        
        epoch_val_loss = running_val_loss / len(val_dataloader.dataset)
        
        print(f"Epoch [{epoch+1}/{NUM_EPOCH}] - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")
    
    return epoch_val_loss

In [4]:
def run_optimization(n_trials=50):
    study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials)
    
    print("Best trial:")
    trial = study.best_trial
    
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    
    # Create the best model
    best_model = create_model(trial)
    
    return best_model, study

In [ ]:
# Run the optimization process
best_model, study = run_optimization(n_trials=20)  # Adjust number of trials as needed

# Optionally save the best model
torch.save(best_model.state_dict(), 'best_recommender_model.pt')

# Optionally plot optimization results
try:
    # Plot optimization history
    plot_optimization_history(study)
    plt.savefig('optimization_history.png')
    
    # Plot parameter importances
    plot_param_importances(study)
    plt.savefig('param_importances.png')
except:
    print("Visualization couldn't be generated. Make sure matplotlib is installed.")

# Final evaluation of the best model
final_val_loss = evaluate(best_model)
print(f"Final validation loss with best model: {final_val_loss:.4f}")

[I 2025-04-03 18:21:27,148] A new study created in memory with name: no-name-7c80d3d1-413d-41d4-a180-7f3320689ecc


Trial 0 - Model parameters: 7818560
Trial 0, Epoch [1/100] - Train Loss: 35.8992, Val Loss: 35.6079
Trial 0, Epoch [2/100] - Train Loss: 35.1610, Val Loss: 34.9291
Trial 0, Epoch [3/100] - Train Loss: 34.4733, Val Loss: 34.1509
Trial 0, Epoch [4/100] - Train Loss: 33.5218, Val Loss: 32.2966
Trial 0, Epoch [5/100] - Train Loss: 31.0952, Val Loss: 27.3284
Trial 0, Epoch [6/100] - Train Loss: 26.7475, Val Loss: 28.1953
Trial 0, Epoch [7/100] - Train Loss: 26.5068, Val Loss: 25.9473
Trial 0, Epoch [8/100] - Train Loss: 26.1347, Val Loss: 27.2563
Trial 0, Epoch [9/100] - Train Loss: 26.8553, Val Loss: 26.6076
Trial 0, Epoch [10/100] - Train Loss: 25.9421, Val Loss: 25.5041
Trial 0, Epoch [11/100] - Train Loss: 24.9949, Val Loss: 25.0348
Trial 0, Epoch [12/100] - Train Loss: 24.5598, Val Loss: 24.3830
Trial 0, Epoch [13/100] - Train Loss: 24.2384, Val Loss: 24.4079
Trial 0, Epoch [14/100] - Train Loss: 24.3095, Val Loss: 24.2411
Trial 0, Epoch [15/100] - Train Loss: 24.2544, Val Loss: 24.178

[I 2025-04-03 18:27:06,808] Trial 0 finished with value: 22.200599670410156 and parameters: {'ratings_embedding_dim': 20, 'ratings_lstm_hidden_size': 57, 'ratings_lstm_num_layers': 8, 'ratings_word_size': 17, 'ratings_final_mlp_factor': 11, 'ratings_embedding_output': 56, 'user_embedding_dim': 29, 'user_embedding_output': 14, 'user_factor': 6, 'expert_factor': 4}. Best is trial 0 with value: 22.200599670410156.


Trial 0, Epoch [100/100] - Train Loss: 21.2917, Val Loss: 22.4179
Trial 1 - Model parameters: 2453099
Trial 1, Epoch [1/100] - Train Loss: 47.1597, Val Loss: 47.7529
Trial 1, Epoch [2/100] - Train Loss: 46.8475, Val Loss: 47.4487
Trial 1, Epoch [3/100] - Train Loss: 46.5474, Val Loss: 47.1533
Trial 1, Epoch [4/100] - Train Loss: 46.2608, Val Loss: 46.8094
Trial 1, Epoch [5/100] - Train Loss: 45.9094, Val Loss: 46.4605
Trial 1, Epoch [6/100] - Train Loss: 45.5694, Val Loss: 46.0437
Trial 1, Epoch [7/100] - Train Loss: 45.1097, Val Loss: 45.1995
Trial 1, Epoch [8/100] - Train Loss: 44.1653, Val Loss: 43.6721
Trial 1, Epoch [9/100] - Train Loss: 42.7798, Val Loss: 42.8218
Trial 1, Epoch [10/100] - Train Loss: 42.1952, Val Loss: 41.0588
Trial 1, Epoch [11/100] - Train Loss: 40.5785, Val Loss: 40.1956
Trial 1, Epoch [12/100] - Train Loss: 39.8568, Val Loss: 39.6190
Trial 1, Epoch [13/100] - Train Loss: 39.4467, Val Loss: 39.4608
Trial 1, Epoch [14/100] - Train Loss: 39.3952, Val Loss: 39.42

[I 2025-04-03 18:32:15,162] Trial 1 finished with value: 39.31455993652344 and parameters: {'ratings_embedding_dim': 5, 'ratings_lstm_hidden_size': 27, 'ratings_lstm_num_layers': 4, 'ratings_word_size': 25, 'ratings_final_mlp_factor': 13, 'ratings_embedding_output': 21, 'user_embedding_dim': 17, 'user_embedding_output': 22, 'user_factor': 13, 'expert_factor': 2}. Best is trial 0 with value: 22.200599670410156.


Trial 1, Epoch [100/100] - Train Loss: 38.9822, Val Loss: 39.6392
Trial 2 - Model parameters: 1279890
Trial 2, Epoch [1/100] - Train Loss: 40.3700, Val Loss: 41.0211
Trial 2, Epoch [2/100] - Train Loss: 39.9979, Val Loss: 40.7082
Trial 2, Epoch [3/100] - Train Loss: 39.7000, Val Loss: 40.4457
Trial 2, Epoch [4/100] - Train Loss: 39.4419, Val Loss: 40.2133
Trial 2, Epoch [5/100] - Train Loss: 39.2185, Val Loss: 39.9972
Trial 2, Epoch [6/100] - Train Loss: 38.9997, Val Loss: 39.7634
Trial 2, Epoch [7/100] - Train Loss: 38.7578, Val Loss: 39.5086
Trial 2, Epoch [8/100] - Train Loss: 38.4973, Val Loss: 39.1700
Trial 2, Epoch [9/100] - Train Loss: 38.1274, Val Loss: 38.7723
Trial 2, Epoch [10/100] - Train Loss: 37.7401, Val Loss: 38.1985
Trial 2, Epoch [11/100] - Train Loss: 37.0355, Val Loss: 37.9873
Trial 2, Epoch [12/100] - Train Loss: 36.7381, Val Loss: 38.0234
Trial 2, Epoch [13/100] - Train Loss: 36.7288, Val Loss: 37.9940
Trial 2, Epoch [14/100] - Train Loss: 36.7419, Val Loss: 37.96